In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pylab as plt
import os
import gc
import seaborn as sns
import tensorflow as tf
import scipy.misc

from os import listdir
from os.path import isfile, join

from skimage.transform import resize

import warnings
print('TF',tf.__version__)
print('NP',np.__version__)

TF 1.13.1
NP 1.16.4


In [0]:
##READING LABELS

labels_path='/content/drive/My Drive/CS231n/Project/Label_train/'
train_data = pd.read_csv(labels_path+'train.csv')


In [0]:
DELF_FOLDER='content/train/delf_features_np_all256/'

In [0]:
import requests
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import threading
import urllib
import cv2
import time

import keras
from keras.applications import ResNet50,InceptionResNetV2
from keras.applications import DenseNet121
from keras import backend as K
from keras import regularizers
from keras.engine.topology import Input
from keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Dense, Flatten, GlobalMaxPooling2D, \
    Lambda, MaxPooling2D, Reshape, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.utils import Sequence
from sklearn.model_selection import train_test_split

from collections import Counter

import os

Using TensorFlow backend.


In [0]:
# checking present DELF feture files

from os import listdir
import sys
from os import listdir
from os.path import isfile, join
defl_dir=DELF_FOLDER
defl_list = [f[:-9] for f in listdir(defl_dir) if isfile(join(defl_dir, f))]
len(defl_list)

In [0]:
# Double check all versus present DELF
defl_list_df = pd.DataFrame(defl_list) 
Checked_df=train_data.set_index('id').join(defl_list_df.set_index(0),how='inner')

In [0]:
NUM_THRESHOLD = 100
train=Checked_df


counts = dict(Counter(train['landmark_id']))
landmarks_dict = {x:[] for x in train.landmark_id.unique() if counts[x] >= NUM_THRESHOLD}
NUM_CLASSES = len(landmarks_dict)
print("Total number of valid classes: {}".format(NUM_CLASSES))

i = 0
landmark_to_idx = {}
idx_to_landmark = []
for k in landmarks_dict:
    landmark_to_idx[k] = i
    idx_to_landmark.append(k)
    i += 1



all_ids=train.index.tolist()

all_landmarks = train['landmark_id'].tolist()


valid_ids_dict = {x[0]:landmark_to_idx[x[1]] for x in zip(all_ids, all_landmarks) if x[1] in landmarks_dict}
valid_ids_list = [x[0] for x in zip(all_ids, all_landmarks) if x[1] in landmarks_dict]

NUM_EXAMPLES = len(valid_ids_list)
print("Total number of valid examples: {}".format(NUM_EXAMPLES))

Total number of valid classes: 117
Total number of valid examples: 20179


In [0]:

train1_ids, test_ids = train_test_split(valid_ids_list, test_size=0.1)#1.2*NUM_CLASSES/NUM_EXAMPLES,random_state=1234)
train_ids, validation_ids = train_test_split(train1_ids, test_size=0.2)#1.2*NUM_CLASSES/NUM_EXAMPLES,random_state=1234)
print(len(train_ids))

14528


In [0]:
# checking average number of returned features

import numpy as np
import glob

dir=DELF_FOLDER
files = sorted(glob.glob(dir + '/*.npy'))
delf_num_features = []
for f in files:
    delf_num_features.append(len(np.load(f)))

print(len(delf_num_features))

ll=[item for item in delf_num_features]
import statistics
print(min(ll), max(ll),statistics.median(ll))

In [0]:
def download_delf(id_):
  feature_path='/content/content/train/delf_features_np_all256/'
  try:
    #print(feature_path+id_+'.delf.npy')
    f=np.load(feature_path+id_+'.delf.npy',allow_pickle=True)
    #print(len(f))
    if len(f)>=150:
      return f[:150].reshape((150,40))
    
    else:
      return np.array([])
    
    
  except:
    return np.array([])
  


In [0]:
valid_x=[]
valid_y=[]

for item in validation_ids:

    valid_xi=download_delf(item)

    valid_yi=valid_ids_dict[item]
    if valid_xi.shape[0]>0:
        valid_x.append(valid_xi)
        valid_y.append(valid_yi)
print(len(valid_x))

y = np.zeros((len(valid_x), NUM_CLASSES))
        
for i in range(len(valid_y)):
    y[i,valid_y[i]] = 1.
valid_y=y
valid_x=np.array(valid_x)

1300


In [0]:

class DataGenDEFL(Sequence):
    def __init__(self, data, batch_size=24, verbose=1):
        self.batch_size=batch_size
        self.data_urls = data

    def normalize(self,data):
        return data
    
    def __getitem__(self, index):
      
        number_of_crops=1
        
        
        #print('Index: ',index)
        batch_urls = random.sample(self.data_urls, int(self.batch_size/number_of_crops))
        
        batch_urls=self.data_urls[index*self.batch_size:(index+1)*self.batch_size]
        
        y_classes = []
        f=np.array([])
        F_list=[]
        
        #url is id
        for url in batch_urls:
            f=download_delf(url)
            
      
            if f.shape[0]>0:
              
              F_list.append(f)
              y_classes.append(valid_ids_dict[url.split("/")[-1]])
            
     
        
        
        F=np.array(F_list)
        y = np.zeros((len(F_list), NUM_CLASSES))
        
        for i in range(len(y_classes)):
            y[i,y_classes[i]] = 1.
        
  
        
        return F,y
            
    def on_epoch_end(self):
        return

    def __len__(self):

        return int(len(train_ids)/self.batch_size)-1
      
def binary_crossentropy_n_cat(y_t, y_p):
    return keras.metrics.binary_crossentropy(y_t, y_p) * NUM_CLASSES

#GeM

gm_exp = tf.Variable(3., dtype=tf.float32)
def generalized_mean_pool_2d(X):
    import tensorflow as tf
    #gm_exp = tf.Variable(3., dtype=tf.float32)
    
    pool = (tf.reduce_mean(tf.abs(X**(gm_exp)), 
                           axis=[1,2], 
                           keepdims=False)+1.e-8)**(1./gm_exp)
    return pool
  
def accuracy_class(y_true, y_pred):
    true = K.argmax(y_true, axis=1)
    pred = K.argmax(y_pred, axis=1)
    matches = K.equal(true, pred)
    return K.mean(matches)
  
def batch_GAP(y_t, y_p):
  
    import tensorflow as tf
    pred_cat = tf.argmax(y_p, axis=-1)    
    y_t_cat = tf.argmax(y_t, axis=-1) * tf.cast(
        tf.reduce_sum(y_t, axis=-1), tf.int64)
    
    n_pred = tf.shape(pred_cat)[0]
    is_c = tf.cast(tf.equal(pred_cat, y_t_cat), tf.float32)

    GAP = tf.reduce_mean(
          tf.cumsum(is_c) * is_c / tf.cast(
              tf.range(1, n_pred + 1), 
              dtype=tf.float32))
    
    return GAP  

In [0]:
#simple model



from keras.layers.normalization import BatchNormalization
from keras.layers import GlobalMaxPooling1D,GlobalAveragePooling1D

input_shape = (150, 40)

X_feat = Input(input_shape)
#Add bottleneck layers (Dense + BN + Activation + Dropout + Output)
                                
X = Dense(NUM_CLASSES)(X_feat)
X=BatchNormalization()(X)
X = Activation('relu')(X)
X = Dropout(0.3)(X)

X = Dense(NUM_CLASSES)(X)
X=BatchNormalization()(X)
X = Activation('relu')(X)
X = Dropout(0.3)(X)

X=Flatten()(X)

X = Dense(NUM_CLASSES, activation='softmax')(X)

model = Model(inputs=X_feat, outputs=X)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 150, 40)           0         
_________________________________________________________________
dense_15 (Dense)             (None, 150, 117)          4797      
_________________________________________________________________
batch_normalization_9 (Batch (None, 150, 117)          468       
_________________________________________________________________
activation_9 (Activation)    (None, 150, 117)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 150, 117)          0         
_________________________________________________________________
dense_16 (Dense)             (None, 150, 117)          13806     
_________________________________________________________________
batch_normalization_10 (Batc (None, 150, 117)          468       
__________

In [0]:
os.chdir("/content/drive/My Drive/CS231n/Project/Data/Train/0/0/0/")
from keras.callbacks import History, CSVLogger,ReduceLROnPlateau,ModelCheckpoint
history = keras.callbacks.History()


model_name='simple_features_0609'

LOG_DIR='/content/drive/My Drive/CS231n/Project/LOG/'
Check_DIR='/content/drive/My Drive/CS231n/Project/Checkpoints/'


csv_logger = CSVLogger(LOG_DIR+'training_'+model_name+'.log')



opt = Adam(0.0001)
#opt = Adam(0.00001)

checkpoint_name=model_name+'.h5'
checkpoint_filepath=Check_DIR+checkpoint_name

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.000001)

checkpoint=keras.callbacks.ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=[accuracy_class,batch_GAP,binary_crossentropy_n_cat])

#check dir
#if os. path. isfile(checkpoint_filepath):
  #model.load_weights('/content/drive/My Drive/CS231n/Project/Checkpoints/resnet_with_crop_0531.h5')
#model.load_weights(checkpoint_filepath)

  #print("Best weights loaded")
  
model.fit_generator(generator=DataGenDEFL(train_ids, batch_size=256),
                    shuffle=True,
                    validation_data=[valid_x, valid_y],
                    epochs=20,
                    use_multiprocessing=True,
                    workers=8,
                    verbose=1,
                    callbacks=[history,csv_logger,reduce_lr#,checkpoint
                              ])




Epoch 1/20
55/55 [==============================] - 10s 178ms/step - loss: 0.0020 - accuracy_class: 0.9930 - batch_GAP: 0.9842 - binary_crossentropy_n_cat: 0.2322 - val_loss: 0.0281 - val_accuracy_class: 0.4354 - val_batch_GAP: 0.2088 - val_binary_crossentropy_n_cat: 3.2857
Epoch 2/20
55/55 [==============================] - 5s 83ms/step - loss: 0.0014 - accuracy_class: 0.9964 - batch_GAP: 0.9918 - binary_crossentropy_n_cat: 0.1672 - val_loss: 0.0280 - val_accuracy_class: 0.4385 - val_batch_GAP: 0.2144 - val_binary_crossentropy_n_cat: 3.2767
Epoch 3/20
55/55 [==============================] - 5s 99ms/step - loss: 0.0012 - accuracy_class: 0.9973 - batch_GAP: 0.9934 - binary_crossentropy_n_cat: 0.1350 - val_loss: 0.0279 - val_accuracy_class: 0.4308 - val_batch_GAP: 0.2021 - val_binary_crossentropy_n_cat: 3.2679
Epoch 4/20
55/55 [==============================] - 5s 96ms/step - loss: 9.4407e-04 - accuracy_class: 0.9969 - batch_GAP: 0.9938 - binary_crossentropy_n_cat: 0.1105 - val_loss: 0.

In [0]:


x_=valid_x
y_=valid_y

def val_gap(x_,y_):

  result=model.predict(x_, batch_size=128, verbose=0, steps=None)

  true_=y_.astype('int64')
  pred_=result

  _,acc=tf.metrics.average_precision_at_k(true_,pred_,1)
  custom_gap=batch_GAP(true_,pred_)

  sess = tf.Session()
  sess.run(tf.local_variables_initializer())

  print('Validation GAP: ',sess.run(acc))
  print('Custom GAP: ', sess.run(custom_gap))

val_gap(x_,y_)


In [0]:
import matplotlib.pyplot as plt

#history
#loss: 0.0086 - accuracy_class
# Plot training & validation accuracy values
plt.plot(history.history['batch_GAP'])
plt.plot(history.history['val_batch_GAP'])
plt.title('Batch GAP')
plt.ylabel('GAP')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

plt.plot(history.history['accuracy_class'])
plt.plot(history.history['val_accuracy_class'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()